In [1]:
import numpy as np
import pandas as pd
from keras.models import  Model
from keras.layers import Dense,Input
from keras.optimizers import Adam
from datetime import  datetime
import itertools
import argparse
import os
import pickle
from sklearn.preprocessing import StandardScaler

In [2]:
def get_data():
    df = pd.read_csv('aapl_msi_sbux.csv')
    return df.values

In [3]:
class ReplayBuffer():
    def __init__(self,obs_dim , act_dim,size): # store state and next state
        self.obs1_buf = np.zeros([size,obs_dim],dtype=np.float32)
        self.obs2_buf = np.zeros([size,obs_dim],dtype=np.float32)
        self.acts_buf = np.zeros(size , dtype=np.uint8)
        self.rews_buf = np.zeros(size , dtype=np.float32)
        self.done_buf = np.zeros(size , dtype=np.uint8)
        self.ptr , self.size , self.max_size = 0,0,size
    def store (self, obs , act , rew  , next_obs, done):
        self.obs1_buf = obs
        self.obs2_buf = next_obs
        self.acts_buf = act
        self.rews_buf = rew
        self.done_buf =done
        self.ptr = (self.ptr+1) % self.max_size
        self.size = min(self.size+1 , self.max_size)
    def smaple_batch(self, batch_size=32):
        idxs = np.random.randint(0,self.size,size=batch_size)
        return dict(s=self,
                    s2=self.obs1_buf[idxs],
                    a=self.obs2_buf[idxs],
                    r=self.acts_buf[idxs],
                    d=self.rews_buf[idxs]
                    )

In [ ]:
def get_scaler(env):
    states = []
    for _i in range(env.n_steps):
        action = np.random.choice(env.action_space)
        state,reward,done,info = env.step(action)
        states.append(state)
        if done:
            break
    scaler = StandardScaler()
    scaler.fit(states)
    return states